<a href="https://colab.research.google.com/github/SonnyK33/test/blob/master/Credit_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
import numpy as np
import pandas as pd
import math

creditID = 0
currentYear = 2020.25
currentQuarter = 2
FCF = ['Revenue','Variable Costs','Fixed Costs','EBITDA', 'Less: Capex', 'Less: Working Cap', 'Less: Taxes', 'FCF', 'Beginning Cash', 'Ending Cash']

class corporateCredit:
    creditID+=1
    
  
    def __init__(self, name, ticker, bbgrating, sector, country):
      self.name = name
      self.ticker = ticker
      self.bbgrating = bbgrating
      self.sector = sector
      self.country = country

    def setCurrentFCF(self, rev, rev_g, var_costs, f_costs, cash, capex_rev, WC_rev, taxes):
      self.CurrentFCF = {'rev': rev, 'rev_g': rev_g, 'var_costs': var_costs, 'f_costs': f_costs, 'cash': cash, 'capex/rev': capex_rev, 'WC/rev': WC_rev, 'tax':taxes}

    

    def makeProj (self, qtrs):
      revArray = np.array([self.CurrentFCF['rev']*(1+self.CurrentFCF['rev_g'])**i for i in range(qtrs+1)],dtype=int)
      vcostsArray = np.array([self.CurrentFCF['var_costs']*(1+self.CurrentFCF['rev_g'])**i for i in range(qtrs+1)],dtype=int)
      fcostsArray = np.full(shape=qtrs+1,fill_value=self.CurrentFCF['f_costs'],dtype=int)
      ebitdaArray = revArray+vcostsArray+fcostsArray
      capexArray = revArray * self.CurrentFCF['capex/rev']
      wcArray = revArray * self.CurrentFCF['WC/rev']
      taxArray = ebitdaArray * self.CurrentFCF['tax']  #to fix --> need interest exp
      fcfArray = ebitdaArray + capexArray + wcArray + taxArray # need interest expense

      begCashArray = np.zeros(qtrs+1,dtype=int)
      begCashArray[0] = self.CurrentFCF['cash']
      endCashArray = np.zeros(qtrs+1,dtype=int)
      endCashArray[0] = fcfArray[0]+begCashArray[0]
      

      for i in range(1,qtrs+1):
        begCashArray[i] = endCashArray[i-1]        
        endCashArray[i] = begCashArray[i] + fcfArray[i]
        
      
      FCF_statement=[revArray,vcostsArray,fcostsArray,ebitdaArray,capexArray,wcArray,taxArray,fcfArray,begCashArray,endCashArray]
      FCF_df=pd.DataFrame(FCF_statement,columns = self.makeDateColumn(qtrs), index = FCF, dtype=int)

      # print(ebitdaArray)
      # print(begCashArray)
      # print(endCashArray)
      

      return FCF_df


      
    def makeDateColumn(self,qtrs):
      yearColumns = ['Q'+str(currentQuarter)+'/'+str(math.floor(currentYear))]
      for x in range (qtrs):
          yearColumns.append('Q'+str(((currentQuarter+x) % 4)+1)+'/'+str(math.floor(currentYear+(x+1)*0.25)))
      return yearColumns


    # def setCapitalStructure(self,cash,firstLien,secondLien,otherDebt,unsecured, rd):
    #   self.cash = cash
    #   self.firstLien = firstLien
    #   self.secondLien = secondLien
    #   self.otherDebt = otherDebt
    #   self.unsecured = unsecured
    #   self.rd = rd
    #   self.totalDebt = self.firstLien + self.secondLien + self.otherDebt + self.unsecured
    #   self.netDebt = self
    #   self.intexp = self.totalDebt * rd

    #def addDebt









a = corporateCredit("Ovintiv", "OVV", "BB+", "E&P", "USA" )
a.setCurrentFCF(726,0.01,-200,-257,100,-0.05,-0.01,-0.2)
a_fcf = a.makeProj(10)
a_fcf
#a.setCapitalStructure(100,100,100,100,0.05,0.20)
#x = pd.DataFrame(a_fcf.iloc[1] + a_fcf.iloc[3])

#a.CurrentFCF['cash']





,Q2/2020,Q3/2020,Q4/2020,Q1/2021,Q2/2021,Q3/2021,Q4/2021,Q1/2022,Q2/2022,Q3/2022,Q4/2022
Revenue,726.00,733.00,740.0,747.00,755.00,763.00,770.0,778.00,786.00,794.00,801.00
Variable Costs,-200.00,-202.00,-204.0,-206.00,-208.00,-210.00,-212.0,-214.00,-216.00,-218.00,-220.00
Fixed Costs,-257.00,-257.00,-257.0,-257.00,-257.00,-257.00,-257.0,-257.00,-257.00,-257.00,-257.00
EBITDA,269.00,274.00,279.0,284.00,290.00,296.00,301.0,307.00,313.00,319.00,324.00
Less: Capex,-36.30,-36.65,-37.0,-37.35,-37.75,-38.15,-38.5,-38.90,-39.30,-39.70,-40.05
Less: Working Cap,-7.26,-7.33,-7.4,-7.47,-7.55,-7.63,-7.7,-7.78,-7.86,-7.94,-8.01
Less: Taxes,-53.80,-54.80,-55.8,-56.80,-58.00,-59.20,-60.2,-61.40,-62.60,-63.80,-64.80
FCF,171.64,175.22,178.8,182.38,186.70,191.02,194.6,198.92,203.24,207.56,211.14
Beginning Cash,100.00,271.00,446.0,624.00,806.00,992.00,1183.0,1377.00,1575.00,1778.00,1985.00
Ending Cash,271.00,446.00,624.0,806.00,992.00,1183.00,1377.0,1575.00,1778.00,1985.00,2196.00
